In [19]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
from datetime import datetime
import numpy as np
# Import the API keys.
from config import api_keys
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

In [20]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs # Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)


In [21]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [22]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
print(len(cities))

617


In [23]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + api_keys['weather_api_key']

In [24]:

# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_desc = city_weather["weather"][0]["description"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # check for a snow key, if so we add it to our row
        try:
            city_snow = city_weather["snow"]["1h"] * 0.0393701 # need to convert mm to inches also
        except:
            city_snow = 0
        try:
            city_rain = city_weather["rain"]["1h"] * 0.0393701 # need to convert mm to inches also
        except:
            city_rain = 0
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Current Description": city_desc,
                          "Snow (Inches)": city_snow,
                          "Rain (Inches)": city_rain})

# If an error is experienced, skip the city.
    except Exception as e:
        print("City not found ¯\_(ツ)_/¯")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Ok its done!")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | norman wells
Processing Record 2 of Set 1 | mahon
Processing Record 3 of Set 1 | bambous virieux
Processing Record 4 of Set 1 | albany
Processing Record 5 of Set 1 | arraial do cabo
Processing Record 6 of Set 1 | lavrentiya
Processing Record 7 of Set 1 | butaritari
Processing Record 8 of Set 1 | isla vista
Processing Record 9 of Set 1 | carnarvon
Processing Record 10 of Set 1 | cherskiy
Processing Record 11 of Set 1 | bilibino
Processing Record 12 of Set 1 | manggar
Processing Record 13 of Set 1 | barrow
Processing Record 14 of Set 1 | mataura
Processing Record 15 of Set 1 | san andres
Processing Record 16 of Set 1 | port hedland
Processing Record 17 of Set 1 | seymchan
Processing Record 18 of Set 1 | ponta delgada
Processing Record 19 of Set 1 | khatanga
Processing Record 20 of Set 1 | punta arenas
Processing Record 21 of Set 1 | nouadhibou
Processing Record 22 of Set 1 | hilo
Processing Record 

In [25]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
new_column_order = ["City", "Country","Lat", "Lng", "Max Temp", "Humidity", "Cloudiness","Wind Speed","Current Description","Rain (Inches)","Snow (Inches)"]
city_data_df = city_data_df[new_column_order]

# set the formatting options for rain/snow
city_data_df['Rain (Inches)'] = city_data_df['Rain (Inches)'].map("{:.1f}".format)
city_data_df['Snow (Inches)'] = city_data_df['Snow (Inches)'].map("{:.1f}".format)


In [26]:
# Converting Snow/Rain rows to float64
city_data_df['Snow (Inches)'] = pd.to_numeric(city_data_df['Snow (Inches)'], errors='coerce')
city_data_df['Rain (Inches)'] = pd.to_numeric(city_data_df['Rain (Inches)'], errors='coerce')


In [27]:
# Create the output file (CSV).
output_data_file = './data/WeatherPy_database.csv'
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")